In [0]:
import numpy as np


In [0]:
!wget https://huseinhouse.com/dataset/text/kerajaan
!wget https://huseinhouse.com/dataset/text/pembangkang
!wget https://huseinhouse.com/dataset/text/apple

--2019-06-29 02:45:35--  https://huseinhouse.com/dataset/text/kerajaan
Resolving huseinhouse.com (huseinhouse.com)... 42.191.206.100
Connecting to huseinhouse.com (huseinhouse.com)|42.191.206.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 623601 (609K) [application/octet-stream]
Saving to: ‘kerajaan’

kerajaan            100%[===================>] 608.99K   521KB/s    in 1.2s    

2019-06-29 02:45:42 (521 KB/s) - ‘kerajaan’ saved [623601/623601]

--2019-06-29 02:45:44--  https://huseinhouse.com/dataset/text/pembangkang
Resolving huseinhouse.com (huseinhouse.com)... 42.191.206.100
Connecting to huseinhouse.com (huseinhouse.com)|42.191.206.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 594342 (580K) [application/octet-stream]
Saving to: ‘pembangkang’

pembangkang         100%[===================>] 580.41K   515KB/s    in 1.1s    

2019-06-29 02:45:49 (515 KB/s) - ‘pembangkang’ saved [594342/594342]

--2019-06-29 02:45:50--  ht

In [0]:
with open('apple') as fopen:
  apple = fopen.readlines()
len(apple)

2296

In [0]:
with open('kerajaan') as fopen:
  kerajaan = fopen.readlines()
len(kerajaan)

6957

In [0]:
with open('pembangkang') as fopen:
  pembangkang = fopen.readlines()
len(pembangkang)

7636

In [0]:
import random
k = 500
apple_sample = random.sample(apple, k)
pembangkang_sample = random.sample(pembangkang, k)
kerajaan_sample = random.sample(kerajaan, k)

apple as class 0, 
pembangkang class 1,
kerajaan class 2

In [0]:
Y = [0]*k + [1]*k + [2]*k
len(Y)
#to generate label

1500

In [0]:
X = apple_sample + pembangkang_sample + kerajaan_sample
len(X) == len(Y)

True

###Basic Text Cleaning

In [0]:
for i in range(len(X)):
  X[i] = X[i].lower()

### Bag of Words

In [0]:
 unique_words = list(set(' '.join(X).split()))
 len(unique_words) #to make bag of words, so we change from set to list

7384

In [0]:
bow = np.zeros((len(X), len(unique_words)))
bow.shape

#1500 sentences times 7348 unique words

(1500, 7384)

In [0]:
set(' '.join(['makan apa tu']).split())

{'apa', 'makan', 'tu'}

In [0]:
# to add no. of frequency of the word, if it shows up 3 times, add 1 for three times
for no, word in enumerate(X):
  for w in word.split():
    bow[no, unique_words.index(w)] +=1
  

In [0]:
unique_words[:100] #if we have 1 word, it represents 1 dimension, if we have 1500 unqie words, we have 1500 dimensions

In [0]:
example_x = ['saya suka makan ayam ayam', 'suka tidur', 'suka menganjing']
example_unique = list(set(' '.join(example_x).split()))
example_unique

['tidur', 'saya', 'ayam', 'menganjing', 'makan', 'suka']

In [0]:
example_bow = np.zeros((len(example_x), len(example_unique)))
example_bow

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [0]:
for no, x in enumerate(example_x):
  for w in x.split():
    example_bow[no, example_unique.index(w)] += 1
    
example_bow

array([[0., 1., 2., 0., 1., 1.],
       [1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 1.]])

In [0]:
df = np.zeros((len(example_unique)))
df

array([0., 0., 0., 0., 0., 0.])

In [0]:
#df document frequency, no of document contains saya

for no, w in enumerate(example_unique):
  for string in example_x:
    if w in string:
      df[no] += 1
      
df

array([1., 1., 1., 1., 1., 3.])

In [0]:
df = len(example_x) / df # N / df

In [0]:
log_df = np.log(df)
log_df


array([1.09861229, 1.09861229, 1.09861229, 1.09861229, 1.09861229,
       0.        ])

In [0]:
example_tfidf = example_bow * log_df
example_tfidf

array([[0.        , 1.09861229, 2.19722458, 0.        , 1.09861229,
        0.        ],
       [1.09861229, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 1.09861229, 0.        ,
        0.        ]])

### Calculate Likelihood

likelihood depends on conditions = possibility (based on the event of A happened)
frequenties divided by sum = probability

In [0]:
classes, counts = np.unique(Y, return_counts = True)
print(classes)
print(counts)

[0 1 2]
[500 500 500]


In [0]:
prob_class = {}

for i in range (len(classes)):
  prob_class[classes[i]] = counts[i]/ len(Y)
  
prob_class

{0: 0.3333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}

In [0]:
np_X = np.array(bow)
np_Y = np.array(Y)

In [0]:
sum_class = {}
for c in classes:
  sum_class[c] = np_X[np_Y == c].sum()
sum_class
  
  #find the bag of words for class 0, class 1, class 2
  #sum of c0 and c1 and c2

{0: 6157.0, 1: 6094.0, 2: 6624.0}

In [0]:
#calculate likelihood
likelihood = np.zeros((len(classes), bow.shape[1]))
likelihood.shape
#row is size of class
#column is size of vocab

(3, 7384)

In [0]:
alpha = 1

In [0]:
for c_ in classes:
  c = np_X[np_Y == c_].sum(axis = 0)
  likelihood[c_] = (alpha + c) / (sum_class[c_] + alpha * bow.shape[1]) #laplace, bow.shape[1] is the no. of unique words
  
likelihood


array([[1.47699579e-04, 7.38497895e-05, 7.38497895e-05, ...,
        7.38497895e-05, 7.38497895e-05, 7.38497895e-05],
       [7.41949844e-05, 2.22584953e-04, 1.48389969e-04, ...,
        7.41949844e-05, 2.96779938e-04, 7.41949844e-05],
       [7.13877784e-05, 7.13877784e-05, 7.13877784e-05, ...,
        1.42775557e-04, 7.13877784e-05, 1.42775557e-04]])

In [0]:
bow.shape, likelihood.shape

((1500, 7384), (3, 7384))

In [0]:
numerator = np.copy(likelihood)
for c in classes:
  numerator[c] = numerator[c] * prob_class[c] # p(x|c) * p(c)
numerator

array([[4.92331930e-05, 2.46165965e-05, 2.46165965e-05, ...,
        2.46165965e-05, 2.46165965e-05, 2.46165965e-05],
       [2.47316615e-05, 7.41949844e-05, 4.94633229e-05, ...,
        2.47316615e-05, 9.89266459e-05, 2.47316615e-05],
       [2.37959261e-05, 2.37959261e-05, 2.37959261e-05, ...,
        4.75918523e-05, 2.37959261e-05, 4.75918523e-05]])

In [0]:
forward = bow.dot(numerator.T)

x_predict = forward/forward.sum(axis = 1, keepdims = True)
x_predict.shape

(1500, 3)

In [0]:
### Check how accuracy of the model

(np.argmax(x_predict, axis = 1) == np_Y).mean()

0.694

In [0]:
new_string = ['saya hate pembangkang']
new_bow = np.zeros((len(new_string), len(unique_words)))

for no, x in enumerate(new_string):
  for w in x.split():
    new_bow[no, unique_words.index(w)] += 1
    
new_bow.sum()

new_feedforward = new_bow.dot(numerator.T)
new_feedforward / new_feedforward.sum(axis=1, keepdims=True)

array([[0.08063408, 0.60758237, 0.31178356]])